# Bad Demo: One-Shot LLM on Raw Data

## ⚠️ WARNING: Anti-Pattern Demonstration

**This notebook intentionally demonstrates the WRONG way to use LLMs for analytics.**

We will:
1. Show raw table schemas to an LLM with no semantic guidance
2. Ask it to write SQL and provide insights in a single shot
3. Watch it produce confident but completely wrong results

### Expected Failure Modes

This approach will fail in multiple ways:
- **Revenue attribution error**: Wrong join path, attributing revenue incorrectly
- **Many-to-many inflation**: Cartesian explosion from improper joins
- **Time window drift**: Inconsistent date ranges across metrics
- **Metric misuse**: Using orders instead of conversions in CAC calculation
- **Dimension ambiguity**: Mixing utm_source and channel, causing duplication

### Business Question

"Which channel mix change is most likely to improve CAC next month, given a recent anomaly in referral traffic?"

In [ ]:
# Setup
import os
import duckdb
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

# Load environment
load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Connect to database
db_path = '../data/synthetic_data.duckdb'
conn = duckdb.connect(db_path, read_only=True)

print("✓ Environment loaded")
print("✓ Database connected")

## Step 1: Raw Table Inventory

Let's show the LLM our raw tables without any semantic guidance about:
- How to join them safely
- Which metrics are canonical
- What time windows to use
- How to attribute revenue

In [ ]:
# Get table schemas
tables = conn.execute("SHOW TABLES").fetchall()
print("Available tables:")
for table in tables:
    print(f"  - {table[0]}")

# Build raw schema description for LLM
schema_description = "Database Schema:\n\n"

for table in tables:
    table_name = table[0]
    columns = conn.execute(f"DESCRIBE {table_name}").fetchall()
    schema_description += f"{table_name}:\n"
    for col in columns:
        schema_description += f"  - {col[0]} ({col[1]})\n"
    schema_description += "\n"

# Show a preview
print("\n" + schema_description[:800] + "...")

## Step 2: One-Shot Prompt Execution

Now we'll ask the LLM to:
1. Write SQL to analyze the business question
2. Provide a confident recommendation

**No guardrails. No semantic layer. No validation.**

In [ ]:
business_question = """Which channel mix change is most likely to improve CAC next month, 
given a recent anomaly in referral traffic?"""

prompt = f"""{schema_description}

Business Question: {business_question}

Write a SINGLE SQL query to analyze this question and provide a recommendation.
The query should calculate CAC by channel and identify opportunities for optimization.

Return your response in this format:
SQL:
[your SQL query]

INSIGHT:
[your recommendation in 2-3 sentences]
"""

# Call LLM
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7
)

llm_output = response.choices[0].message.content
print("LLM Response:")
print("=" * 80)
print(llm_output)

In [ ]:
# Extract and execute the SQL
import re

# Parse SQL from response
sql_match = re.search(r'SQL:(.*?)(?:INSIGHT:|$)', llm_output, re.DOTALL)
if sql_match:
    generated_sql = sql_match.group(1).strip()
    # Remove markdown code blocks if present
    generated_sql = re.sub(r'^```sql\s*', '', generated_sql)
    generated_sql = re.sub(r'^```\s*', '', generated_sql)
    generated_sql = re.sub(r'```\s*$', '', generated_sql)
    generated_sql = generated_sql.strip()
    
    print("\nExecuting generated SQL...\n")
    print(generated_sql)
    print("\n" + "=" * 80)
    
    try:
        result_df = conn.execute(generated_sql).df()
        print("\nQuery Results:")
        print(result_df)
        
        # Store for failure analysis
        bad_results = result_df
        bad_sql = generated_sql
    except Exception as e:
        print(f"\nQuery failed: {e}")
        bad_results = None
        bad_sql = generated_sql
else:
    print("Could not parse SQL from LLM response")
    bad_results = None
    bad_sql = None

## Step 3: Failure Exhibit

Let's analyze what went wrong with the LLM-generated SQL.

In [ ]:
# Check for specific failure patterns
failures = []

if bad_sql:
    sql_lower = bad_sql.lower()
    
    # Failure 1: Revenue attribution error
    if 'fact_orders' in sql_lower and 'fact_sessions' not in sql_lower:
        failures.append({
            'failure': 'Revenue Attribution Error',
            'description': 'Joined fact_orders directly to dim_campaigns without going through fact_sessions',
            'impact': 'Revenue is incorrectly attributed; many orders will be dropped or duplicated',
            'correct': 'Must use: fact_orders → fact_sessions → dim_campaigns (last-touch attribution)'
        })
    
    # Failure 2: Many-to-many inflation
    if 'fact_sessions' in sql_lower and 'fact_ad_spend' in sql_lower:
        if sql_lower.count('join') >= 2 and 'session_id' not in sql_lower:
            failures.append({
                'failure': 'Many-to-Many Cartesian Explosion',
                'description': 'Joined fact_sessions and fact_ad_spend on campaign_id without proper grain',
                'impact': 'Row counts multiply incorrectly; metrics are inflated by 10-100x',
                'correct': 'These tables must be aggregated separately before joining, or use semantic layer'
            })
    
    # Failure 3: Time window drift
    date_filters = re.findall(r'(date|timestamp).*?interval.*?(\d+)', sql_lower)
    if len(set([d[1] for d in date_filters])) > 1:
        failures.append({
            'failure': 'Time Window Drift',
            'description': 'Different tables use different date ranges (e.g., 30 days for spend, 90 days for revenue)',
            'impact': 'CAC calculation mixes mismatched time periods; results are meaningless',
            'correct': 'All metrics must use the same canonical window (default: 90 days)'
        })
    
    # Failure 4: Metric misuse
    if 'order_id' in sql_lower and ('cac' in sql_lower or 'acquisition' in sql_lower):
        failures.append({
            'failure': 'Metric Definition Error',
            'description': 'Used orders in CAC calculation instead of conversions',
            'impact': 'CAC is understated because not all sessions convert to orders',
            'correct': 'CAC = spend / conversions, where conversions come from fact_sessions.converted_flag'
        })
    
    # Failure 5: Dimension ambiguity
    if 'utm_source' in sql_lower or 'source' in sql_lower:
        failures.append({
            'failure': 'Dimension Ambiguity',
            'description': 'Mixed dim_campaigns.channel with utm_source or other dimension',
            'impact': 'Channels are inconsistently defined; totals don\'t match',
            'correct': 'Must use canonical dimension: dim_campaigns.channel only'
        })

print("\n" + "=" * 80)
print("FAILURE ANALYSIS")
print("=" * 80)

for i, failure in enumerate(failures, 1):
    print(f"\n{i}. {failure['failure']}")
    print(f"   What happened: {failure['description']}")
    print(f"   Impact: {failure['impact']}")
    print(f"   Correct approach: {failure['correct']}")

if not failures:
    print("\nNo obvious failures detected in pattern matching, but the results are still likely wrong!")
    print("This demonstrates another problem: it's hard to even detect when one-shot LLM SQL fails.")

## Step 4: The Overconfident Narrative

Despite these errors, the LLM produced a confident recommendation:

In [ ]:
# Extract insight
insight_match = re.search(r'INSIGHT:(.*)', llm_output, re.DOTALL)
if insight_match:
    insight = insight_match.group(1).strip()
    print("LLM Recommendation:")
    print("=" * 80)
    print(insight)
    print("=" * 80)
    print("\n⚠️ This recommendation is based on WRONG DATA due to the failures above.")
    print("⚠️ Implementing this could waste budget and harm business performance.")

## Step 5: Post-Mortem Summary

### What We Learned

**Single LLM calls on raw data fail because:**

1. **No semantic contracts**: The LLM doesn't know the canonical metric definitions
2. **No join validation**: Complex schemas allow many wrong join paths
3. **No time window enforcement**: Different metrics drift to different periods
4. **No grain management**: Many-to-many joins cause silent data explosions
5. **Overconfidence**: The LLM has no way to know it's wrong

### The Right Way

The solution requires:
- **Semantic layer**: Canonical metrics with tested SQL templates
- **Modular agents**: Small, focused, testable components
- **Deterministic logic**: Use LLMs only for ambiguity resolution
- **Observability**: Log every decision and SQL execution
- **Testing**: Validate joins, grains, and results

### Next Steps

See `02_good_modular_dspy.ipynb` for the correct implementation using:
- DSPy agent architecture
- Semantic layer from `config/semantic.yml`
- Reproducible, testable, explainable analytics

In [ ]:
# Cleanup
conn.close()
print("\n✓ Demo complete. Database connection closed.")
print("\nRemember: This is what NOT to do. See the good demo for the right approach.")